<a href="https://colab.research.google.com/github/dhruvthakur2000/Customer-Supporter/blob/main/Customer_Support_Ticket_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
import os

warcoder_customer_support_ticket_tagging_path = kagglehub.dataset_download('warcoder/customer-support-ticket-tagging')

print('Data source import complete.')
print(f'Dataset downloaded to: {warcoder_customer_support_ticket_tagging_path}')

# Construct the full path to the CSV file using the downloaded path
csv_file_path = os.path.join(warcoder_customer_support_ticket_tagging_path, 'customer_tickets.csv')



Data source import complete.
Dataset downloaded to: /kaggle/input/customer-support-ticket-tagging


In [ ]:
#!pip install transformers datasets torch scikit-learn

## Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

In [ ]:
df = pd.read_csv(csv_file_path)
df.columns = ["text","labels"]
df.head()

,text,labels
0,"Dear Customer Support Team, We are experiencin...",Technical Support
1,"Dear Customer Support,<br><br>I hope this mess...",Product Support
2,"Dear Tech Online Store Customer Support,\n\nI ...",Returns and Exchanges
3,"Dear IT Services Customer Support, \n\nWe are ...",Product Support
4,"Greetings IT Services Customer Support,\n\nI a...",Technical Support


In [ ]:
df.dropna(inplace=True)

In [ ]:
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels']) # converts labels which are in character to numerix format

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'labels']])
hf_dataset = dataset.train_test_split(test_size=0.145)
print(hf_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 288
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 50
    })
})


In [ ]:
label_encoder.classes_

array(['Billing and Payments', 'Customer Service', 'General Inquiry',
       'Human Resources', 'IT Support', 'Product Support',
       'Returns and Exchanges', 'Sales and Pre-Sales',
       'Service Outages and Maintenance', 'Technical Support'],
      dtype=object)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "microsoft/deberta-v3-base"  # loading the deberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)
#The function returns the output of the tokenizer, which will be a numerical representation
# of the text,including input IDs, attention masks, and possibly other information needed by the model.
tokenized_datasets = hf_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=0.00002,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,#regularzation to prevent overfitting
    logging_dir='./logs',
    logging_steps=30,
    report_to='none'
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    processing_class=tokenizer
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
30,2.083100,1.811931
60,1.828000,1.768443
90,1.767800,1.678632
120,1.656800,1.568313
150,1.573800,1.530387
180,1.430500,1.458313
210,1.296700,1.408849
240,1.183400,1.377445


Step,Training Loss,Validation Loss
30,2.083100,1.811931
60,1.828000,1.768443
90,1.767800,1.678632
120,1.656800,1.568313
150,1.573800,1.530387
180,1.430500,1.458313
210,1.296700,1.408849
240,1.183400,1.377445
270,1.101300,1.441707
300,1.015900,1.378209


TrainOutput(global_step=360, training_loss=1.4045861191219753, metrics={'train_runtime': 15670.6796, 'train_samples_per_second': 0.184, 'train_steps_per_second': 0.023, 'total_flos': 538768242754560.0, 'train_loss': 1.4045861191219753, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("./text-classification-model")
tokenizer.save_pretrained("./text-classification-model")

In [ ]:
from transformers import pipeline
# loading the locally saved model
classifier = pipeline("text-classification", model="./text-classification-model", tokenizer=tokenizer)

In [ ]:
# Predictor Function to evaluate some tickets
def predictor(input_ticket,org_label):
    print(f"Input Ticket: {input_ticket}")
    result = classifier(input_ticket)
    print("\n")
    org_label_decoded = label_encoder.inverse_transform([int(org_label)])[0]
    decoded_label = label_encoder.inverse_transform([int(result[0]['label'].split("_")[-1])])[0]
    print("Original Label: ",org_label,",Original Label Decoded: ",org_label_decoded)
    print(f"Predicted Label: {int(result[0]['label'].split('_')[-1])} ,Predicted label Decoded: {decoded_label}")

In [ ]:
predictor(df['text'][319],df['labels'][319])

In [ ]:
predictor(df['text'][338],df['labels'][338])

In [ ]:
predictor(df['text'][317],df['labels'][317])

In [ ]:
predictor(df['text'][210],df['labels'][210])

In [ ]:
predictor(df['text'][50],df['labels'][50])

In [ ]:
predictor(df['text'][175],df['labels'][175])

In [ ]:
predictor(df['text'][15],df['labels'][15])